# 5. lekce - Cvičení 1: Pivot tabulky

## 1. Dvě kritéria

*Poznámka:* Zadání tohoto příkladu ti možná bude připadat povědomé, bylo již v lekci o podmíněném výběru. Pokud jsi ale příklad neřešil(a), nevadí. Pokud ano, uvidíš, že řešení bude díky funkci `pivot()` mnohem jednodušší.

Připravujeme seznam potravin pro účely lékařského výzkumu, který se bude zabývat kardiovaskulárním systémem. Chceme vybrat potraviny, které splňují dvě kritéria:

- nízký obsah nasycených mastných kyselin (`"Fatty acids, total saturated"`, uvažuj méně než 1 gram),
- vysoký obsah vlákniny (`"Fiber, total dietary"`, uvažuj více než 5 gramů).

Zatímco nasycené mastné kyseliny jsou považovány za spíše škodlivé pro kardiovaskulární systém, vláknina je považována spíše za prospěšnou. Vyber z tabulky `food_nutrient_pivot` potraviny, které vyhovují oběma podmínkám.

In [ ]:
# Příprava dat z lekce, pokud ještě nemáme
import pandas as pd

food_nutrient = pd.read_csv("data/food_nutrient.csv")
duplicates = food_nutrient[food_nutrient.duplicated(subset=['fdc_id', 'name'], keep=False)]
food_nutrient = food_nutrient.drop_duplicates(subset=["fdc_id", "name"])

food_nutrient_pivot = pd.pivot(food_nutrient, index="fdc_id", columns="name", values="amount")

branded_food = pd.read_csv("data/branded_food.csv")
food = pd.concat([pd.read_csv("data/food_sample_100.csv"), pd.read_csv("data/food_other.csv")], ignore_index=True)
food_brands = pd.merge(food, branded_food, on="fdc_id")
food_nutrient_pivot = pd.merge(food_nutrient_pivot, food_brands, on="fdc_id")

# Náhled dat
display(food_nutrient.head(3))
display(food_nutrient_pivot.head(3))
food_nutrient_pivot.shape

Chceme vybrat potraviny, které splňují dvě kritéria:
- nízký obsah nasycených mastných kyselin (`"Fatty acids, total saturated"`, uvažuj méně než 1 gram),
- vysoký obsah vlákniny (`"Fiber, total dietary"`, uvažuj více než 5 gramů).

In [ ]:
# Výběr dat podle kritérií
food_nutrient_pivot_healthy = food_nutrient_pivot[
    (food_nutrient_pivot["Fatty acids, total saturated"] < 1) 
    & (food_nutrient_pivot["Fiber, total dietary"] > 5)
    ]

# Pro srovnání, takto vypadal výběr dat v 2. lekci bez předzpracování pomocí pivot:
# food_nutrient_filtered = food_nutrient[
#     ((food_nutrient["name"] == "Fatty acids, total saturated") & (food_nutrient["amount"] < 1)) 
#     | ((food_nutrient["name"] == "Fiber, total dietary") & (food_nutrient["amount"] > 4))
#     ]

food_nutrient_pivot_healthy

# 5. lekce - Cvičení 2: Další pivot funkce

## 1. Booking

Stáhni si data ze souboru o rezervacích hotelů ze serveru Booking.com. Data jsou uložená v souboru [booking.csv](data/booking.csv). U rezervací evidujeme, jestli byly zrušené, to najdeme ve sloupci `is_canceled` (0 pro zrušené rezervace a 1 pro nezrušené).

In [342]:
import pandas as pd

# Načtení dat
data1_all = pd.read_csv("data/hotel_bookings.csv")
data1_all.head()

,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,deposit_type,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date
0,Resort Hotel,0,342,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
1,Resort Hotel,0,737,2015,July,27,1,0,0,2,...,No Deposit,NaN,NaN,0,Transient,0.0,0,0,Check-Out,2015-07-01
2,Resort Hotel,0,7,2015,July,27,1,0,1,1,...,No Deposit,NaN,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
3,Resort Hotel,0,13,2015,July,27,1,0,1,1,...,No Deposit,304.0,NaN,0,Transient,75.0,0,0,Check-Out,2015-07-02
4,Resort Hotel,0,14,2015,July,27,1,0,2,2,...,No Deposit,240.0,NaN,0,Transient,98.0,0,1,Check-Out,2015-07-03


Vytvoř kontingenční tabulku, která porovná počet zrušených rezervací podle typu hotelu (sloupec `hotel`). Je více rezervací zrušeno pro městské hotely nebo pro hotely v rezortech?

In [345]:
# Crosstab pro typ hotelu
pd.crosstab(data1_all["hotel"], data1_all["is_canceled"])

# To stejné pomocí pivot_table()
# (aggfunc="size" spočítá počet výskytů každé kombinace)
# pd.pivot_table(data=data1_all, index="hotel", columns="is_canceled", aggfunc="size")

is_canceled,0,1
hotel,,
City Hotel,46228,33102
Resort Hotel,28938,11122



Dále zkus rezervace rozdělit do skupin podle toho, v jakém předstihu byly rezervace provedeny. Zaměř se pouze na rezervace v městských hotelech, tj. vytvoř tabulku, která bude obsahovat pouze data, které mají ve sloupci `hotel` hodnotu `City Hotel`- Využij sloupec `lead_time`. Níže máš skupiny, podle kterých můžeš data rozdělit.

| Lead Time              | Reservation Category    |
|------------------------|-------------------------|
| 0-7                    | Last-minute             |
| 8-30                   | Short-term              |
| 31-180                 | Medium-term             |
| 180-inf                | Long-term               |


In [346]:
# Výběr pouze dat týkajících se City Hotelu
data1 = data1_all[data1_all['hotel'] == 'City Hotel']

# Příprava proměnných pro kategorizaci
bins = [0, 7, 30, 180, float('inf')]
labels = ['Last-minute', 'Short-term', 'Medium-term', 'Long-term']

# Přidání nového sloupce pro kategorie rezervací na základě jejich předstihu
data1['lead_time_category'] = pd.cut(data1['lead_time'], bins=bins, labels=labels)

data1.sample(5)

C:\Users\mapup\AppData\Local\Temp\ipykernel_2864\3528596715.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data1['lead_time_category'] = pd.cut(data1['lead_time'], bins=bins, labels=labels)


,hotel,is_canceled,lead_time,arrival_date_year,arrival_date_month,arrival_date_week_number,arrival_date_day_of_month,stays_in_weekend_nights,stays_in_week_nights,adults,...,agent,company,days_in_waiting_list,customer_type,adr,required_car_parking_spaces,total_of_special_requests,reservation_status,reservation_status_date,lead_time_category
64945,City Hotel,1,56,2017,March,12,21,0,3,2,...,86.0,NaN,0,Transient,85.0,0,0,Canceled,2017-01-24,Medium-term
97291,City Hotel,0,346,2016,September,38,13,0,2,2,...,6.0,NaN,0,Transient-Party,115.0,0,1,Check-Out,2016-09-15,Long-term
55817,City Hotel,1,171,2016,August,35,22,1,1,3,...,9.0,NaN,0,Transient,137.7,0,0,Canceled,2016-03-04,Medium-term
91767,City Hotel,0,192,2016,June,26,24,0,2,1,...,34.0,NaN,0,Transient-Party,95.0,0,0,Check-Out,2016-06-26,Long-term
81543,City Hotel,1,218,2016,June,24,9,0,2,2,...,133.0,NaN,0,Transient,135.0,0,0,Canceled,2015-12-14,Long-term


Vytvoř si pivot tabulku, která zobrazuje počty rezervací v jednotlivých kategoriích v závislosti na tom, jestli byly zrušeny nebo ne. Pro které kategorie je více zrušených rezervací a pro které naopak více nezrušených? A v jaké skupině je celkově nejvíce rezervací?

In [347]:
# Crosstab
data1_pivot = pd.crosstab(data1["lead_time_category"], data1["is_canceled"])
# data1_pivot = pd.crosstab(index=data1["lead_time_category"], columns=data1["is_canceled"])
data1_pivot

is_canceled,0,1
lead_time_category,,
Last-minute,6653,1046
Short-term,8674,3880
Medium-term,21974,17046
Long-term,6091,10857


In [348]:
# Bonus: parametr margins=True přidá součty řádků a sloupců
data1_pivot = pd.crosstab(data1["lead_time_category"], data1["is_canceled"], margins=True)
data1_pivot

is_canceled,0,1,All
lead_time_category,,,
Last-minute,6653,1046,7699
Short-term,8674,3880,12554
Medium-term,21974,17046,39020
Long-term,6091,10857,16948
All,43392,32829,76221


In [349]:
# Crosstab s normalizací dat - hodnoty vyjadřují podíl zrušených versus nezrušených v rámci kategorie (např. 86 % vs 14 %)
data_pivot1 = pd.crosstab(data1["lead_time_category"], data1["is_canceled"], normalize="index")
data_pivot1

is_canceled,0,1
lead_time_category,,
Last-minute,0.864138,0.135862
Short-term,0.690935,0.309065
Medium-term,0.563147,0.436853
Long-term,0.359393,0.640607


In [351]:
# Bonus: Zaokrouhlení hodnot v tabulce pro větší přehlednost
# data_pivot1.round(2)

# Případně přepočet na procenta
data_pivot1.round(2) * 100

is_canceled,0,1
lead_time_category,,
Last-minute,86.0,14.0
Short-term,69.0,31.0
Medium-term,56.0,44.0
Long-term,36.0,64.0


## 2. Titanic a další pivot tabulky

K tomuto cvičení využij data o cestujících na Titanicu ze souboru [titanic.csv](data/titanic.csv).

In [ ]:
import pandas as pd

# Načtení dat
data2 = pd.read_csv("data/titanic.csv")
data2.head()

Vytvoř další kontingenční tabulku, která porovná závislost mezi třídou (sloupec `Pclass`), ve které cestoval, a tím, jestli přežil. Zkus spočítat počty přeživších z každé třídy. 

In [ ]:
# Crosstab podle cestovní třídy
data2_pivot = pd.crosstab(data2["Pclass"], data2["Survived"])
data2_pivot

Dále zkus vypočítat relativní počet přeživších pro jednotlivé třídy.

In [ ]:
# Crosstab podle cestovní třídy s normalizací dat - hodnoty vyjadřují podíl přeživších v rámci třídy (např. 37 % vs 63 %)
data_pivot2 = pd.crosstab(data2["Pclass"], data2["Survived"], normalize="index")
data_pivot2

Dále zkus vytvořit tabulku s relativním počtem přeživších v závislosti na pohlaví (sloupec `Sex`).

In [ ]:
# Crosstab podle pohlaví
data_pivot2 = pd.crosstab(data2["Sex"], data2["Survived"])
data_pivot2

In [ ]:
# Crosstab podle pohlaví s normalizací dat - hodnoty vyjadřují podíl přeživších v rámci pohlaví (např. 26 % vs 74 %)
data_pivot2 = pd.crosstab(data2["Sex"], data2["Survived"], normalize="index")
data_pivot2

## 3. Doprava

Stáhni si data ze souboru [traffic.csv](data/traffic.csv), který zobrazuje počet aut v hodinových intervalech na čtyřech různých křižovatkách.

Pro následující dva úkoly využij funkci `cut`.

- Tabulka je poměrně velká, zkusme tedy rozdělit den na několik částí v závislosti na hodině měření (sloupec `Hour`):
  - noc (22 hodin až 5 hodin),
  - ranní špička (6 hodin až 9 hodin),
  - poledne (10 hodin až 13 hodin),
  - odpolední špička (14 hodin až 16 hodin),
  - večer (17 hodin až 21 hodin).
- Dny v týdnu (sloupec `DayOfWeek`) rozděl na pracovní dny a víkend. Nejprve je třeba se ujistit, které číslo znamená který den. Podívej se například první záznam. Tam je datum 2015-11-01, tj 11. listopad 2015. Ve sloupci `DayOfWeek` je hodnota 6. Podívej se do kalendáře, který den v týdnu to je. Od jakého čísla jsou dny v týdnu v tabulce počítané?


Nakonec pomocí pivot tabulky průměrné počty aut (sloupec `Vehicles`) v jednotlivých částech dne a typech dne.

Tipy:

- Všimni si, že noc je rozdělená na dvě části - od půlnoci do 5 hodin a od 22 hodin do půlnoci. Bohužel musí být seznam názvů skupin unikátní, můžeš ale například vytvořit skupiny `noc_1` a `noc_2`. Poté můžeš podřetězce `_1` a `_2` odstranit pomocí metody `.str.replace()`.

In [ ]:
import pandas as pd
import numpy as np

# Načtení dat
data3 = pd.read_csv("data/traffic.csv")
data3.head()

In [ ]:
# Příprava proměnných pro kategorizaci
bins = [0, 5, 9, 13, 16, 21, 24]
labels = ["noc_1", "ranní špička", "poledne", "odpolední špička", "večer", "noc_2"]

# Přidání sloupce s označením kategorie dat podle denní doby
data3['HourGroup'] = pd.cut(data3["Hour"], bins=bins, labels=labels)

# Úprava názvů kategorií noc_1 a noc_2 na noc
data3["HourGroup"] = data3["HourGroup"].str.replace("_1", "")
data3["HourGroup"] = data3["HourGroup"].str.replace("_2", "")

In [ ]:
# Příprava proměnných pro kategorizaci
bins = [1, 5, 6]
labels = ["weekday", "weekend"]

# Přidání sloupce s označením kategorie dat podle dne v týdnu
data3['DayOfWeekGroup'] = pd.cut(data3["DayOfWeek"], bins=bins, labels=labels)

data3.head(15)

In [ ]:
# Vytvoření pivot tabulky pro průměrné počty aut v týdnu a o víkendu
data_pivot3 = pd.pivot_table(data3, values="Vehicles", index="Hour", columns="DayOfWeekGroup", aggfunc=np.mean, margins=True)
data_pivot3

## 4. Půjčování kol

V souboru [london_merged.csv](data/london_merged.csv) najdeš informace o počtu vypůjčení jízdních kol v Londýně. Vytvoř kontingenční tabulku, zobrazí průměrný počet reze (sloupec `weather_code` se sloupcem udávající měsíc). Počet vypůjčených kol je ve sloupci `cnt`. V datech je dále sloupec `Hour`, který označuje hodinu, pro který počet máme. Použij kód z předchozí části a rozděl data do kategorií podle denní doby. Vytvoř pivot tabulku, která zobrazí průměrné počty vypůjčených kol v závislosti na typu počasí (sloupec `weather_code`) a denní době.

Definice jednotlivých kódů ze sloupce `weather_code` jsou:

- 1 = Jasno; includes minor atmospheric conditions like haze, fog, patches of fog, or fog in the vicinity.
- 2 = Částečně oblačno; indicating a sky that is partly cloudy with scattered or few clouds.
- 3 = Převážně oblačno; mostly covered with clouds, but not entirely overcast.
- 4 = Zataženo; indicating a fully overcast or cloudy sky.
- 7 = Déšť / Lehký déšť; covering light rain conditions, including light rain showers.
- 10 = Déšť s bouřkou; rain accompanied by a thunderstorm.
- 26 = Sněžení; general condition of snow falling.
- 94 = Mrznoucí mlha; indicating fog conditions where the water droplets in the fog freeze upon contact with surfaces.

Pokud ti zobrazení pomocí číselných kódů přijde nepřehledné, můžeš kódy nahradit popisy. K tomu lze využít metodu `map`. Pro metodu `map` je nutné mít tzv. slovník, tj. strukturu, která obsahuje data ve dvojicích. Hodnoty ve dvojici jsou odděleny dvojtečkou a celá dvojice je oddělená čárkou. V našem případě máme slovník `weather_code_to_czech`, kde máme číselný kód a popisek hodnoty pro každé počasí. Výsledek poté uložíme do sloupce `weather_description`. Vyzkoušej kód níže a podívej se, zda skutečně v tabulce přibyl sloupec `weather_description` a zda hodnoty odpovídají číselným kódům.

```py
weather_code_to_czech = {
    1: "Jasno",
    2: "Částečně oblačno",
    3: "Převážně oblačno",
    4: "Zataženo",
    7: "Déšť / Lehký déšť",
    10: "Déšť s bouřkou",
    26: "Sněžení",
    94: "Mrznoucí mlha"
}
data['weather_description'] = data['weather_code'].map(weather_code_to_czech)
```

In [ ]:
import pandas as pd
import numpy as np

# Načtení dat
data4 = pd.read_csv('data/london_merged.csv')
data4.head()
# data4.info()

In [ ]:
# Převod data z řetězce na datový typ časové značky (timestamp)
data4['timestamp'] = pd.to_datetime(data4['timestamp'])

# Přidání sloupce s měsíci
data4['month'] = data4['timestamp'].dt.month

# data4['timestamp'].info()

In [ ]:
# Převedení číselných kódů na slovní označení
weather_code_to_czech = {
    1: "Jasno",
    2: "Částečně oblačno",
    3: "Převážně oblačno",
    4: "Zataženo",
    7: "Déšť / Lehký déšť",
    10: "Déšť s bouřkou",
    26: "Sněžení",
    94: "Mrznoucí mlha"
}

data4['weather_description'] = data4['weather_code'].map(weather_code_to_czech)

In [ ]:
# Přidání sloupců s označením kategorie dat podle denní doby (stejně jako ve 3. příkladu)
bins = [0, 5, 9, 13, 16, 21, 24]
labels = ["noc_1", "ranní špička", "poledne", "odpolední špička", "večer", "noc_2"]

data4['HourGroup'] = pd.cut(data4["Hour"], bins=bins, labels=labels)
data4["HourGroup"] = data4["HourGroup"].str.replace("_1", "")
data4["HourGroup"] = data4["HourGroup"].str.replace("_2", "")

In [ ]:
# Vytvoření pivot tabulky
pivot_table_weather = pd.pivot_table(data4, values='cnt', index='weather_description', columns="HourGroup", aggfunc=np.mean)
pivot_table_weather